In [2]:
import numpy as np
import cv2
from glob import glob
import random
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Input, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from tensorflow.keras import Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils  import to_categorical
from tensorflow.keras.metrics import AUC, Recall, Precision, BinaryAccuracy
from sklearn.model_selection import train_test_split

In [3]:
import os
import tensorflow as tf

class DataGen(tf.keras.utils.Sequence):
    def __init__(self, ids, path, batch_size=8, image_size=256):
        self.ids = ids
        self.path = path
        self.batch_size = batch_size
        self.image_size = image_size
        self.on_epoch_end()
        
    def __load__(self, id_name):
        ## Path
        image_path = os.path.join(self.path, id_name)
        
        ## Reading Image
        image = cv2.imread(image_path, 1)
        #image = cv2.resize(image, (self.image_size, self.image_size))
        
        _, w, _ = np.shape(image)
        
        w = int(w/2)
        mask = image[:, w:, :]
        im = image[:, :w, :]

        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

        ## Normalizaing 
        im = im/255.0
        mask = mask/255.0
        
        return im, mask
    
    def __getitem__(self, index):
        if(index+1)*self.batch_size > len(self.ids):
            self.batch_size = len(self.ids) - index*self.batch_size
        
        files_batch = self.ids[index*self.batch_size : (index+1)*self.batch_size]
        
        image = []
        mask  = []
        
        for id_name in files_batch:
            _img, _mask = self.__load__(id_name)
            image.append(_img)
            mask.append(_mask)
            
        image = np.array(image)
        mask  = np.array(mask)
        
        return image, mask
    
    def on_epoch_end(self):
        pass
    
    def __len__(self):
        return int(np.ceil(len(self.ids)/float(self.batch_size)))

In [4]:
test_path = f'./c0_vips/c0_vips_files/17/'
test_ids = os.listdir(test_path)


for image in test_ids:

    print(image.split('.')[0])

0_0
0_1
0_10
0_100
0_101
0_102
0_103
0_104
0_105
0_106
0_107
0_108
0_109
0_11
0_110
0_111
0_112
0_113
0_114
0_115
0_116
0_117
0_118
0_119
0_12
0_120
0_121
0_122
0_123
0_124
0_125
0_126
0_127
0_128
0_129
0_13
0_130
0_131
0_132
0_133
0_134
0_135
0_136
0_137
0_138
0_139
0_14
0_140
0_141
0_142
0_143
0_144
0_145
0_146
0_147
0_148
0_149
0_15
0_150
0_151
0_152
0_153
0_154
0_155
0_156
0_157
0_158
0_159
0_16
0_160
0_161
0_162
0_163
0_164
0_165
0_166
0_167
0_168
0_169
0_17
0_170
0_171
0_172
0_173
0_174
0_175
0_176
0_177
0_178
0_179
0_18
0_180
0_181
0_182
0_183
0_184
0_185
0_186
0_187
0_188
0_189
0_19
0_190
0_191
0_192
0_193
0_194
0_195
0_196
0_197
0_198
0_199
0_2
0_20
0_200
0_201
0_202
0_203
0_204
0_205
0_206
0_207
0_208
0_209
0_21
0_210
0_211
0_212
0_213
0_214
0_215
0_216
0_217
0_218
0_219
0_22
0_220
0_221
0_222
0_223
0_224
0_225
0_226
0_227
0_228
0_229
0_23
0_230
0_231
0_232
0_233
0_234
0_235
0_236
0_237
0_238
0_239
0_24
0_240
0_241
0_242
0_243
0_244
0_245
0_246
0_247
0_248
0_249
0_25
0_250
0_

In [5]:
epochs = 5
batch_size = 4
image_size = 256

train_path = "./train"
val_path = "./val"

train_ids = os.listdir(train_path)
valid_ids = os.listdir(val_path)

train_gen = DataGen(train_ids, train_path, image_size=image_size, batch_size=batch_size)
valid_gen = DataGen(valid_ids, val_path, image_size=image_size, batch_size=batch_size)

train_steps = len(train_ids)//batch_size
valid_steps = len(valid_ids)//batch_size

#Testing
im_train = train_gen.__getitem__(0)
im_val = valid_gen.__getitem__(0)
print(np.shape(im_train[0]))
print(np.shape(im_val[0]))

(4, 256, 256, 3)
(4, 256, 256, 3)


In [6]:
def down_sample(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    p = tf.keras.layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p

def up_sample(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = tf.keras.layers.UpSampling2D((2, 2))(x)
    concat = tf.keras.layers.Concatenate()([us, skip])
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c



In [7]:
def UNet():
    f = [16, 16, 32, 64, 128, 256]
    inputs = tf.keras.layers.Input((image_size, image_size, 3))
    
    p0 = inputs
    c1, p1 = down_sample(p0, f[0]) #128 -> 64
    print(c1.shape)
    c2, p2 = down_sample(p1, f[1]) #64 -> 32
    print(c2.shape)
    c3, p3 = down_sample(p2, f[2]) #32 -> 16
    print(c3.shape)
    c4, p4 = down_sample(p3, f[3]) #16->8
    print(c4.shape)
    c5, p5 = down_sample(p4, f[4]) #16->8
    print(p5.shape)

    bn = bottleneck(p5, f[5])
    print(bn.shape)

    u0 = up_sample(bn, c5, f[4]) #8 -> 16
    print(u0.shape)
    u1 = up_sample(u0, c4, f[3]) #8 -> 16
    u2 = up_sample(u1, c3, f[2]) #16 -> 32
    u3 = up_sample(u2, c2, f[1]) #32 -> 64
    u4 = up_sample(u3, c1, f[0]) #64 -> 128
    
    outputs = tf.keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u2)
    model = tf.keras.models.Model(inputs, outputs)
    return model

model = UNet()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])
model.summary()

(None, 256, 256, 16)
(None, 128, 128, 16)
(None, 64, 64, 32)
(None, 32, 32, 64)
(None, 8, 8, 128)
(None, 8, 8, 256)
(None, 16, 16, 128)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 16  448         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 256, 16  2320

In [8]:
def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    p = tf.keras.layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = tf.keras.layers.UpSampling2D((2, 2))(x)
    concat = tf.keras.layers.Concatenate()([us, skip])
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def UNet():
    f = [16, 16, 32, 64, 128, 256]
    inputs = tf.keras.layers.Input((image_size, image_size, 3))
    
    p0 = inputs
    c1, p1 = down_block(p0, f[0]) #128 -> 64
    c2, p2 = down_block(p1, f[1]) #64 -> 32
    c3, p3 = down_block(p2, f[2]) #32 -> 16
    c4, p4 = down_block(p3, f[3]) #16->8
    c5, p5 = down_block(p4, f[4]) #16->8

    bn = bottleneck(p5, f[5])

    u0 = up_block(bn, c5, f[4]) #8 -> 16
    u1 = up_block(u0, c4, f[3]) #8 -> 16
    u2 = up_block(u1, c3, f[2]) #16 -> 32
    u3 = up_block(u2, c2, f[1]) #32 -> 64
    u4 = up_block(u3, c1, f[0]) #64 -> 128
    
    outputs = tf.keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u4)
    model = tf.keras.models.Model(inputs, outputs)
    return model

model = UNet()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_23 (Conv2D)             (None, 256, 256, 16  448         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_24 (Conv2D)             (None, 256, 256, 16  2320        ['conv2d_23[0][0]']              
                                )                                                           

In [9]:
model.fit(train_gen, validation_data=valid_gen, steps_per_epoch=train_steps, validation_steps=valid_steps, 
                    epochs=epochs, verbose=1)

Epoch 1/5
1367/1367 [==============================] - 577s 420ms/step - loss: 0.0818 - acc: 0.9706 - val_loss: 0.0239 - val_acc: 0.9899
Epoch 2/5
1367/1367 [==============================] - 293s 214ms/step - loss: 0.0322 - acc: 0.9874 - val_loss: 0.0073 - val_acc: 0.9972
Epoch 3/5
1367/1367 [==============================] - 293s 214ms/step - loss: 0.0242 - acc: 0.9902 - val_loss: 0.0063 - val_acc: 0.9975
Epoch 4/5
1367/1367 [==============================] - 299s 219ms/step - loss: 0.0189 - acc: 0.9923 - val_loss: 0.0067 - val_acc: 0.9977
Epoch 5/5
1367/1367 [==============================] - 316s 231ms/step - loss: 0.0177 - acc: 0.9928 - val_loss: 0.0045 - val_acc: 0.9983


In [10]:
class DataGen(tf.keras.utils.Sequence):
    def __init__(self, ids, path, batch_size=8, image_size=256):
        self.ids = ids
        self.path = path
        self.batch_size = batch_size
        self.image_size = image_size
        self.on_epoch_end()
        
    def __load__(self, id_name):
        ## Path
        image_path = os.path.join(self.path, id_name)
        
        ## Reading Image
        image = cv2.imread(image_path, 1)
        #image = cv2.resize(image, (self.image_size, self.image_size))
        
        _, w, _ = np.shape(image)
        
        w = int(w)
        #mask = image[:, w:, :]
        im = image[:, :w, :]

        #mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

        ## Normalizaing 
        im = im/255.0
        #mask = mask/255.0
        mask = 0
        
        return im, mask
    
    def __getitem__(self, index):
        if(index+1)*self.batch_size > len(self.ids):
            self.batch_size = len(self.ids) - index*self.batch_size
        
        files_batch = self.ids[index*self.batch_size : (index+1)*self.batch_size]
        
        image = []
        mask  = []
        
        for id_name in files_batch:
            _img, _mask = self.__load__(id_name)
            image.append(_img)
            mask.append(_mask)
            
        image = np.array(image)
        mask  = np.array(mask)
        
        return image, mask
    
    def on_epoch_end(self):
        pass
    
    def __len__(self):
        return int(np.ceil(len(self.ids)/float(self.batch_size)))

In [14]:
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model\assets


INFO:tensorflow:Assets written to: saved_model/my_model\assets


In [12]:
test_path = f'./c0_vips/c0_vips_files/17/'
test_ids = os.listdir(test_path)

batch_size = 1
test_gen = DataGen(test_ids, test_path, image_size=image_size, batch_size=batch_size)

for i, image in enumerate(test_ids):
    x, y= test_gen.__getitem__(i)
    print(np.shape(x))

    if np.shape(x) == (1, 256, 256, 3):
        predictions = model.predict(x)

        im = np.array(predictions[0]*255, dtype='uint8')
        im = cv2.cvtColor(im, cv2.COLOR_GRAY2BGR)
        #gt = cv2.cvtColor(np.array(y[0, :, :]*255, dtype='uint8'), cv2.COLOR_GRAY2BGR)
        #inp = np.array(x[0, :, :]*255, dtype='uint8')
        #im = cv2.hconcat((gt, im))
        #im = cv2.hconcat((inp, im))
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        image_name = image.split('.')[0]
        print(image_name)
        cv2.imwrite(f'./test_c0/{image_name}.jpeg', im)
        #plt.xlabel('Stained Image                 Weak Labels                   U-net CNN Pred')

        #plt.imshow(im)

(1, 256, 256, 3)
1/1 [==============================] - 0s 209ms/step
0_0
(1, 256, 256, 3)
1/1 [==============================] - 0s 55ms/step
0_1
(1, 256, 256, 3)
1/1 [==============================] - 0s 57ms/step
0_10
(1, 256, 256, 3)
1/1 [==============================] - 0s 67ms/step
0_100
(1, 256, 256, 3)
1/1 [==============================] - 0s 76ms/step
0_101
(1, 256, 256, 3)
1/1 [==============================] - 0s 74ms/step
0_102
(1, 256, 256, 3)
1/1 [==============================] - 0s 69ms/step
0_103
(1, 256, 256, 3)
1/1 [==============================] - 0s 71ms/step
0_104
(1, 256, 256, 3)
1/1 [==============================] - 0s 69ms/step
0_105
(1, 256, 256, 3)
1/1 [==============================] - 0s 66ms/step
0_106
(1, 256, 256, 3)
1/1 [==============================] - 0s 58ms/step
0_107
(1, 256, 256, 3)
1/1 [==============================] - 0s 80ms/step
0_108
(1, 256, 256, 3)
1/1 [==============================] - 0s 58ms/step
0_109
(1, 256, 256, 3)
1/1 [=======

KeyboardInterrupt: 